In [4]:
from popoff.fitting_code import FitModel
import popoff.fitting_output as output
from popoff.input_checker import setup_error_checks
from scipy import optimize
import numpy as np

In [5]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': False }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': -2.0}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': 15.999}
params['potentials'] = {'Li-O': [663.111, 0.119, 0.0],
                        'Ni-O': [1393.540, 0.218, 0.000],
                        'O-O': [25804.807, 0.284, 0.0]}

In [6]:
fit_data = FitModel.collect_info(params, np.array([0]), supercell=None)

Found elements: ['Li', 'Ni', 'O']


/Users/lmm82/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pymatgen/io/vasp/outputs.py:1014: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/lmm82/Documents/current_work/popoff/vaspruns
  warnings.warn("No POTCAR file with matching TITEL fields" " was found in {}".format(os.path.abspath(p)))


In [7]:
fit_data.lammps_data[0]

In [8]:
from lammps import lammps

In [9]:
#Creates a lammps instance
lmp = lammps() 

#Sets unis and atom style
lmp.command('units metal')
lmp.command('atom_style full')

#reads in file
lmp.command('read_data lammps/coords1.lmp')

In [10]:
#gets cores groups
lmp.command('group cores type {}'.format(fit_data.lammps_data[0].type_core()))

#set rigid commands
lmp.command('pair_style buck/coul/long 10.0')
lmp.command('pair_coeff * * 0 1 0')

#set universal commands
lmp.command('kspace_style ewald 1e-6')
lmp.command('min_style cg')

In [11]:
lmp.command('run 0')

In [12]:
# returns the stress tensors?
lmp.command('thermo_style custom temp press')
lmp.command('compute 1 all temp')
lmp.command('compute 2 all pressure 1')

In [13]:
temp = lmp.extract_compute("thermo_temp",0,0)
print("Temperature from compute =",temp)

Temperature from compute = 0.0


In [35]:
thermo_press = lmp.extract_compute("thermo_press", 0, 1)
ip_stresses = np.array([thermo_press[i]/1000 for i in range(0,6,1)])
ip_stresses

array([-1730.52279437, -1678.00742235, -2114.74373876,     9.127073  ,
          20.92300351,  -251.873981  ])

In [38]:
lmp.get_thermo('vol')

1543.29284703092

In [ ]:
    lmp.box.lengths = MagicMock(return_value = [10.0, 10.0, 10.0])
    lmp.box.volume = MagicMock(return_value= 1000.0)